# BEE 4750 Homework 3: Dissolved Oxygen and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to implement a model for dissolved oxygen in a
    river with multiple waste releases and use this to develop a
    strategy to ensure regulatory compliance.
-   Problem 2 asks you to use Monte Carlo simulation to assess how well
    your strategy from Problem 1 performs under uncertainty.
-   Problem 3 (5750 only) asks you to identify where a third discharge
    should be placed to maintain regulatory compliance.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using Random
using Plots
using LaTeXStrings
using Distributions
using CSV
using DataFrames

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

A river which flows at 6 km/d is receiving waste discharges from two
sources which are 15 km apart. The oxygen reaeration rate is 0.55
day<sup>-1</sup>, and the decay rates of CBOD and NBOD are are 0.35 and
0.25 day<sup>-1</sup>, respectively. The river’s saturated dissolved
oxygen concentration is 10m g/L.

If the characteristics of the river inflow and waste discharges are
given in <a href="#tbl-river" class="quarto-xref">Table 1</a>, write a
Julia model to compute the dissolved oxygen concentration from the first
wastewater discharge to an arbitrary distance `d` km downstream. Use
your model to compute the maximum dissolved oxygen concentration up to
50km downstream and how far downriver this maximum occurs.

| Parameter | River Inflow | Waste Stream 1 | Waste Stream 2 |
|:--:|---:|---:|---:|
| Inflow | 100,000 m<sup>3</sup>/d | 10,000 m<sup>3</sup>/d | 15,000 m<sup>3</sup>/d |
| DO Concentration | 7.5 mg/L | 5 mg/L | 5 mg/L |
| CBOD | 5 mg/L | 50 mg/L | 45 mg/L |
| NBOD | 5 mg/L | 35 mg/L | 35 mg/L |

Table 1: River inflow and waste stream characteristics for Problem 1.

**In this problem**:

-   Plot the dissolved oxygen concentration from the first waste stream
    to 50m downriver. What is the minimum value in mg/L?
-   What is the minimum level of treatment (% removal of organic waste)
    for waste stream 1 that will ensure that the dissolved oxygen
    concentration never drops below 4 mg/L, assuming that waste stream 2
    remains untreated? How about if only waste stream 2 is treated?
-   Suppose you are responsible for designing a waste treatment plan for
    discharges into the river, with a regulatory mandate to keep the
    dissolved oxygen concentration above 4 mg/L. Discuss whether you’d
    opt to treat waste stream 2 alone or both waste streams equally.
    What other information might you need to make a conclusion, if any?

#### Problem 1 Model Setup
##### Model Assumptions
- This river is a Prescriptive model because we are asked to specify, or prescribe, an action (in the form of a waste treatment plan) in order to optimize the river flow model to ensure its' DO concentration stays above 4 mg/L.
- DO concentration cannot be negative, and this was implemented in the model by not allowing the DO concentration variable to dip below a value of 0.0. 
- Assume steady state and decay is linear with respect to mass. 
- Assume the river has a flow velocity of 6 [km/day], which receives waste discharge from 2 sources.
- Assume that Random, Plots, LaTexStrings, Distributions, CSV, and DataFrames packages may be utilized in hw03.
- The source begins on the left side at an assumed starting distance of 0 [km] (for model simplicity, i.e. enhanced and accurate analysis for the system), and assigned two variables to the two wastewater discharge sources. The two wastewater discharges are CBOD and NBOD, according to the Problem 1 statement. 
- Based on the Problem 1 statement, and Table 1: River inflow and waste stream characteristics for Problem 1, it is reasonable to assume that CBOD and NBOD both enter the river inflow in waste streams 1 and 2.
- Problem statement clearly says,"two sources which are 15 km apart," but does not define a variable for distance of the river inflow to the 1st Waste Stream source. Therefore, it is reasonable to assume that the simulation will run over a numerically unspecified distance (arbitrary distance) of 0 to D [km] to reach the first Waste Stream source. Then, the distance from the first Waste Stream source to the second Waste Stream source is assumed to be D + 15 [km].This effect of accounting for variable simulation distances is specified analytically and symbolically in the following sections. 
    -   - For this particular scenario, a distance of D = 8 [km] was chosen to be the distance at which the Waste Stream 1 source discharges a flow at a rate of 10000 [m^(3)/day]. This parameter allows for the river model to begin simulating over a step size of 0.05 [km] before additional flow is discharged into the river by Waste Stream 1.
- The problem statement section "In this problem:" states that the dissolved oxygen must be plotted from "the first waste stream to 50m downriver." However, it is reasonable to assume that the author of this problem intended for 'm' to actually be units of [km]. Therefore, in this model, a value of 50 [km] was used for the river flow simulation, and for the plot with a range of 0 [km] to 50 [km].

##### Decision Variables (Units and Meaning)
- **Miscalleneous acronyms/variables used in this Problem, for enhanced clarity:**
    - DO = dissolved oxygen
    - CBOD = Carbonaceous Biochemical Oxygen Demand 
    - NBOD = Nitrogenous Biochemical Oxygen Demand
    - D = distance at which Waste Stream 1 source inflow begins [km]
    - d = the arbitrary distance from Waste Stream 1, used to compute the DO concentration [km].
    - range_distance = 0:0.05:50 = the total distance from the river inflow to the end of the simulation distance at 50 km.
        - Used to account for the river inflow DO, CBOD, and NBOD interactions.
    - range_distance_1_50 = D:0.05:50 = the total distance range from Waste Stream 1 source inlet to the end of the simulation distance at 50 km.
        - Used to plot the DO concentration from the distance at Waste Stream 1 source distance D [km] to the end of the simulation distance at 50 [km], and then find the minimum concentration over this distance and the distance at which the minimum occurs.
- **River inflow:**
    - River Inflow rate, flow_rate_r = 100000 [m^(3)/day] 
    - River Inflow DO concentration, conc_do_r = 7.5 [mg/L]
    - River Inflow CBOD concentration, conc_cbod_r = 5 [mg/L]
    - River Inflow NBOD concentration, conc_nbod_r = 5 [mg/L]
    - River flow velocity, flow_velocity_r = 6 [km/day]
- **Oxygen raeration rate:** rate_ox_reaeration = 0.55 [day^(-1)]
- **CBOD exponential decay rate:** decay_cbod = 0.35 [day^(-1)]
- **NBOD exponential decay rate:** decay_nbod = 0.25 [day^(-1)]
 - **Waste Stream 1 source:**
    - Source 1 Inflow rate, flow_rate_1 = 10000 [m^(3)/day]
    - Source 1 DO Concentration, conc_do_1 = 5 [mg/L]
    - Source 1 CBOD concentration, conc_cbod_1 = 50 [mg/L]
    - Source 1 NBOD concentration, conc_nbod_1 = 35 [mg/L]
- **Waste Stream 2 source:**
    - Source 2 Inflow rate, flow_rate_2 = 15000 [m^(3)/day]
    - Source 2 DO Concentration, conc_do_2 = 5 [mg/L]
    - Source 2 CBOD concentration, conc_cbod_2 = 45 [mg/L]
    - Source 2 NBOD concentration, conc_nbod_2 = 35 [mg/L]

##### Derivation for Objective 
- **Objective:** The primary goal of the Problem 1 model is to derive and implement a model for determining the dissolved oxygen (DO) concentration in a river at various distances, accounting for 2 Waste Stream sources that discharge multiple waste types. Then, use this to develop a strategy to ensure regulatory compliance for keeping the regulatory limit on DO concentration above 4 [mg/L]. On a broader environmental level, it is necessary to ensure the regulatory limit on DO concentration above 4 [mg/L] is upheld so that aquatic ecosystems with organisms and respective habitats can survive, and hypoxia does not occur.
    - Component 1: The initial river indflow DO concentration is simulated and can be influenced by CBOD or NBOD concentrations that are in the water at the beginning of the modeled distance. All three concentrations are given as input parameters, and are implemented in later model components.
    - Component 2: Initialize two Waste Stream sources at the locations D [km] (Waste Stream 1) and D + 15 [km] (Waste Stream 2). Account for the DO, CBOD, and NBOD concentration from  Waste Stream 1 at distance D, onwards through the length of the river. Also account for the DO, CBOD, and NBOD concentration from Waste Stream 2, starting at a distance of D + 15 [km] to analyze how the variations of CBOD and NBOD can affect DO depletion in the river flow model. 
    - Component 3: Implement the effects of oxygen reaeration on the system as it changes with distance, which involved atmospheric oxygen being reabsorbed into the river water. The rate at which oxygen reaeration occurs is proportional to the difference between the DO saturation concentration [mg/L] and the observed DO concentration [mg/L] at each simulation step-size (where step size = 0.05 [km]). 
- **Goal:**
    - The overarching objective of this simulation is to simulate the river flow model to determine DO concentration at an arbitrary distance d [km] downstream, the maximum DO concentration up to 50 km downstream, and how far downriver this maximum concentration occurs. 
    - Plot the DO concentration from the first waste stream to 50 [km] downriver, find its minimum value [mg/L]. Find the minimum level of treatment for Waste Stream 1 to ensure that DO concentration never falls below 4 mg/L, assuming Waste Stream 2 continues not to be treated, and the opposite case as well. 
    - Develop a strategy for ensuring the DO concentration of the river does not follow below the regulatory limit of 4 [mg/L] based on the results of Component 4. Furthermore, discuss how the results would influence a choice between potential waste water treatment plans for the river. 
- **Steps to achieve Objective**:
    - Derive equations for DO concentration [mg/L] as a function of distance for the river flow model.
    - Simulate the model over a distance up to 50 [km].
    - Plot the DO concentration [mg/L] and identify distances at which treatment may be needed to ensure regulatory compliance of keeping the DO concentration above 4 [mg/L].

##### Mathematical Derivation for Objective
- **CBOD and NBOD decay functions:**
    - CBOD_river(distance D [km]) [mg/L] = CBOD inflow concentration [mg/L] * e^((-CBOD decay rate * distance D [km])/(river flow velocity [km/day]))
    - NBOD_river(distance D [km]) [mg/L] = NBOD inflow concentration [mg/L] * e^((-NBOD decay rate * distance D [km])/(river flow velocity [km/day]))
        - Intepretation of CBOD and NBOD decay function: accounts for the exponential decay of CBOD and NBOD to calculate CBOD and NBOD concentrations at a distance D in the river flow model. 
- **Depletion of DO equation:**
    - d(DO depletion(distance D [km])) = CBOD_river(distance D [km]) [mg/L] + NBOD_river(distance D [km]) [mg/L] 
        - Intepretation: calculates DO depletion at each simulated step size of 0.05 [km], accounting for CBOD and NBOD decay.
- **Reaeration of DO equation:**
    - d(DO reaeration(distance D [km])) = reaeration rate * (saturation DO concentration [mg/L] - observed DO concentration(distance D [km]) [mg/L]) * ((1 - e^(-reaeration rate * distance D [km])))
        - Interpretation: DO replenishing rate in the river water.
- **Mass balance for variations in DO concentration in the river:**
    - dDO/dD = (((-CBOD decay rate * CBOD conentration(distance D [km])) + (-NBOD decay rate * NBOD conentration(distance D [km]))) / (river flow rate [m^(3)/day] + Waste Stream 1 flow rate [m^(3)/day] + Waste Stream 2 flow rate [m^(3)/day])) + (reaeration rate * (saturation DO concentration [mg/L] - observed DO concentration(distance D [km]) [mg/L]) * (1 - e^(-reaeration rate * distance D [km])))
        - Interpretation of mass balance for variations in DO concentration in the river derivation: this mass balance allows me, the waste treatment plan designer, to evaluate the variations in DO concentrations with distance downriver, considering the effects of oxygen depletion through CBOD and NBOD decay, as well as the oxygen reaeration rate that occurs proportionally with the difference between the DO saturated and current DO concentrations.
- **Mass balance for  DO concentration at a distance D:**
    - DO concentration(distance D [km]) [mg/L] = Initial DO concentration [mg/L] - d(DO depletion(distance D [km])) + d(DO reaeration(distance D [km]))
        - Where Initial DO concentration [mg/L] = initial DO concentration [mg/L] before Waste Stream 1 inflow, and DO concentration(distance D [km]) [mg/L] is the concentration observed at a distance in the length of the simulated river.
- **Mass balance for final DO concentration**:
    - DO_final(D) = (river DO concentration [mg/L]* river flow rate [m^3/day] + Waste Stream 1 DO concentration [mg/L]* Waste Stream 1 flow rate [m^3/day] + Waste Stream 2 DO concentration [mg/L]* Waste Stream 2 flow rate [m^3/day])/((river flow rate [m^(3)/day] + Waste Stream 1 flow rate [m^(3)/day] + Waste Stream 2 flow rate [m^(3)/day]))

##### Constraints 
- Steady state system
- Regulatory limit on DO, where DO > 4 [mg/L]
    - Interpretation: to ensure safety aquatic ecosystems and comply with environmental standards, specifically for the modeled river for Problem 1.
- River flow rate = 100000 [m^(3)/day], Waste Stream 1 flow rate = 10000 [m^(3)/day], and Waste Stream 2 flow rate = 15000 [m^(3)/day]
    - Cannot change for initial model simulation.
- Reaeration rate = 0.55 [day^(-1)], CBOD decay rate = 0.35 [day^(-1)], NBOD decay rate = 0.25 [day^(-1)], and saturation DO concentration = 10 [mg/L] 
     - Where the decay rate is assumed linear with respect to mass.
- Distance between Waste Stream 1 and Waste Stream 2 = 15 [km], total simulation distance = 50 [km]
     - Cannot change for initial model simulation.

##### Final Answer
- The maximum concentration of DO in the river = 4.6565612446030045[mg/L]
- The maximum concentration of DO in the river occurs at a distance = 41.95[km]
- The minimum concentration of DO in the river = 0.0[mg/L]
- The minimum concentration of DO in the river occurs at a distance = 0.0[km] 

##### Specific Reference to Modeling Results
- **Step-size choice:** Chose a step size of 0.05 [km] for range_distance = 0:0.05:50. Rather than choosing a step size in integral form, such as 1 or 5, a step size of 0.05 was chosen to provide a highly comprehensive, thorough representation of the river flow model to show how the DO concentration varies with distance from 0 to 50 km in the river.
- **Minimum DO concentration and plot discussion:** Based on the output of the model, as detailed in the Final Answer section, the minimum DO concentration = 0.0 [mg/L] and occurs at a distance D = 0.0 [km]. The reasoning for this is outlined in the discussion of the plot below:
    - The plot of Dissolved Oxygen Concentration Variations over River Distance, with an independent variable Distance D [km] and a dependant variable, Dissolved Oxygen Concentration [mg/L] presents an interesting representation of the river flow model. First, the DO concentration [mg/L] from the ranges of D = (10,20) and D = (23,35) on the graph resulted in the constraint that DO concentration cannot be negative, and as such, are represented as a value of 0.0 [mg/L] in these ranges. Furthermore, a sharp increase and decrease occurs at D = 20 and D = 23, which signals a fast drop in DO concentration. This change is supported by Waste Stream 2 inflow beginning at D = 23 [km] (since Waste Stream 1 began at 8 [km], and 8 + 15 [km] = 23 [km]). The significance of this steep increase and decrease provides insights into the adjustment period for the river when accounting for Waste Stream 2's inflow DO, NBOD, and CBOD concentrations. Furthermore, the graph begins to increase in a convex manner at D = 35 [km]. Therefore, after adjusting rapidly to the Waste Stream 2 inflow, in which the DO concentration returns to 0, DO then begins and continues to accumulate with no additional inflows.
- **Maximum DO Concentration discussion:** Furthermore, the printed output shows that the maximum DO concentration = 4.6565612446030045[mg/L] and occurs at a distance = 41.95[km]. This maximum represents an inflection point, and the regulatory limit is upheld at this point since 4.6565612446030045 [mg/L] is greater than 4.0 [mg/L]. 
- **Rectification based on modeling results:** it would be wise to investigate the possibility of changing factors which influence DO in order to protect aquatic life, prevent hypoxia, and ensure the regulatory limit of >4 [mg/L] is upheld. For instance, some factors influencing DO include temperature, pressure, salinity, and plant and microbial life. For this waste treatment plan design, a treatment consisting of temperature decrease could be utilized in Waste Stream 1 and Waste Stream 2 in order to ensure that DO concentration stays above 4 [mg/L]. It is most essential to treate Waste Stream 1 as DO concentrations drop to approximately 0 [mg/L] before reaching Waste Stream 2. Such a minimum is dangerous for the river flow model, and this is also shown by the drop to 0 [mg/L] at the Waste Stream 2 inlet. According to The United States Environmental Protection Agency (https://www.epa.gov/caddis/dissolved-oxygen), higher temperatures decrease oxygen solubility in water, which then lessens the amount of DO that can be held by the water. This reinforcing feedback means that when the opposite is done, (i.e. decreasing the river inflow, Waste Stream 1, and Waste Stream 2 water temperatures), the solubility is thereby increased, which increases the amount of DO that can be held by the water. An additional chemical treatment would still be necessary to prevent CBOD and NBOD decay in Waste Stream 1 and 2 inlets, as the DO concentration may not accumulate enough by the respective inlets to allow for the regulatory limit of >4 [mg/L] to be upheld. 

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using Random
using Plots
using LaTeXStrings
using Distributions
using CSV
using DataFrames

# Initialize function to calculate DO based on relevant decision variables
function do_simulate(d, velocity_r, velocity_ws1, velocity_ws2, C_r, C_ws1, C_ws2, B_r, B_ws1, B_ws2, N_r, N_ws1, N_ws2, ka, kn, kc, Cs, U, 
    discharge_dist_ws1, discharge_dist_ws2)

    # Initialize conditions for distance d along simulated river flow model, separate by river inflow, Waste Stream 1 source, and Waste Stream 2 source
    # For when d hasn't reached Waste Stream 1 source
    if d < discharge_dist_ws1
        # Calculate flow, no sum, just river velocity since Waste Stream sources haven't begun 
        Q = velocity_r # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD, just for river conditions since Waste Stream sources haven't begun
        C_i = C_r
        B = B_r
        N = N_r

    # After Waste Stream 1 source and before Waste Stream 2 discharge source
    elseif d < discharge_dist_ws2
        # Calculate flow, sum respective river and Waste Stream 1 velocitys
        Q = velocity_r + velocity_ws1 # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD based on river and Waste Stream 1 sources over total flow over d < discharge_dist_ws2
        C_i = (C_r * velocity_r + C_ws1 * velocity_ws1) / Q
        B = (B_r * velocity_r + B_ws1 * velocity_ws1) / Q
        N = (N_r * velocity_r + N_ws1 * velocity_ws1) / Q

    # Waste Stream 2 source to end of distance d along river
    else
        # Calculate flow, sum of all inflow source velocitys
        Q = velocity_r + velocity_ws1 + velocity_ws2 # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD now for all 3 inflow sources
        C_i = (C_r * velocity_r + C_ws1 * velocity_ws1 + C_ws2 * velocity_ws2) / Q
        B = (B_r * velocity_r + B_ws1 * velocity_ws1 + B_ws2 * velocity_ws2) / Q
        N = (N_r * velocity_r + N_ws1 * velocity_ws1 + N_ws2 * velocity_ws2) / Q
    end

    # Calculate CBOD and NBOD concentrations, account for exponential decay
    B_decayed = B * exp(-kc * (d - min(d, discharge_dist_ws1)) / U)  # For after Waste Stream 1
    N_decayed = N * exp(-kn * (d - min(d, discharge_dist_ws1)) / U)  # For after Waste Stream 1

    # Calculate DO, after accounting for oxygen reaeration
    a1 = exp(-ka * d / U) 
    a2 = (kc / (ka - kc)) * (exp(-kc * (d / U)) - exp(-ka * (d / U)))
    a3 = (kn / (ka - kn)) * (exp(-kn * (d / U)) - exp(-ka * (d / U)))

    # Calculate final DO concentration and return concentrations
    C_f = Cs * (1 - a1) + (C_i * a1) - (B_decayed * a2) - (N_decayed * a3)
    return C_f, B_decayed, N_decayed
end

# Set river and waste stream properties
velocity_r = 100000 # [m^(3)/day]
velocity_ws1 = 10000 # [m^(3)/day]
velocity_ws2 = 15000 # [m^(3)/day]

# DO concentrations
C_r = 7.5 # [mg/L]
C_ws1 = 5 # [mg/L]
C_ws2 = 5 # [mg/L]

# CBOD concentrations
B_r = 5 # [mg/L]
B_ws1 = 50 # [mg/L]
B_ws2 = 45 # [mg/L]

# NBOD concentrations
N_r = 5 # [mg/L]
N_ws1 = 35 # [mg/L]
N_ws2 = 35 # [mg/L]

# Model parameters
ka = 0.55 # [day^(-1)]
kc = 0.35 # [day^(-1)]
kn = 0.25  # [day^(-1)]
Cs = 10 # [mg/L]
U = 6 # [km/day]

# Initialize Waste Stream 1 and Waste Stream 2 source distances
discharge_dist_ws1 = 15 # [km]
discharge_dist_ws2 = 30 # [km]

# Initialize arbitrary distance to be length of river flow model
d = 0:1:50 # [km]

# Simulate full river flow model
do_simulated = [do_simulate(y, velocity_r, velocity_ws1, velocity_ws2,  C_r, C_ws1, C_ws2, 
    B_r, B_ws1, B_ws2, N_r, N_ws1, N_ws2, ka, kn, kc, Cs, U, discharge_dist_ws1, 
    discharge_dist_ws2) for y in d]

# Initialize arrays for DO, CBOD, and NBOD concentrations to plot simulated results
C_f = [d[1] for d in do_simulated]
B = [d[2] for d in do_simulated]
N = [d[3] for d in do_simulated]

# Calculate minimum DO concentration observed, plot as line later
C_min = minimum([d[1] for d in do_simulated])
return C_min

# Plot DO, CBOD, and NBOD concentrations after simulation iteration
using Plots
p1 = plot(d, C_f, ylabel="DO (mg/L)", xlabel="Distance (km)", label="DO", linewidth=3, color=:black)
plot!(p1, d, B, label="CBOD Concentration", linestyle=:dash, linewidth=3, color=:green)
plot!(p1, d, N, label="NBOD Concentration", linestyle=:dash, linewidth=3, color=:blue)

# Plot DO concentration regulatory standard and minimum DO concentration value
hline!([4], color=:green, label="Regulatory Standard: DO > 4 [mg/L]") # Green, follows regulatory standard
hline!([C_min], color=:red, label="Minimum DO Concentration") # Red, below regulatory standard
display(p1)

### Problem 2 (20 points)

The simplest climate model involves capturing changes to the Earth’s
energy budget (it is commonly called the *energy balance model*, or
EBM). These changes are also called *radiative forcings* (RF), and can
result from several causes, including greenhouse gas emissions, volcanic
eruptions, and changes to the solar cycle. The EBM treats the Earth as a
0-dimensional sphere covered with water, which absorbs heat in response
to radiative forcings. Chanwith global temperature changes resulting
from imbalances in the average (over the entire surface area) heat flux.

The EBM equations are:

$$
\begin{align*}
\overbrace{\frac{dH}{dt}}^{\text{change in heat}} &= \overbrace{F}^{\substack{\text{radiative} \\ \text{forcing}}} - \overbrace{\lambda T}^{\substack{\text{change in} \\ \text{temperature}}} \\
\underbrace{C}_{\substack{\text{ocean heat} \\ \text{capacity}}} \frac{dT}{dt} &= F - \lambda T \\
c\underbrace{d}_{\substack{\text{ocean} \\ \text{mixing depth}}} \frac{dT}{dt} &= F - \lambda T,
\end{align*}
$$

where $c = 4.184\times 10^6 \mathrm{J/K/m}^2$ is the specific heat of
water per area, $d$ is the depth of the ocean mixed layer (we’ll assume
$d = 86 \mathrm{m}$), and $\lambda$ is the **climate feedback factor**
and controls how much the Earth warms in response to increased radiative
forcing (assume
$\lambda = 2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2$)). The total
radiative forcing $F = F_\text{non-aerosol} + \alpha F_\text{aerosol}$,
where $\alpha$ is an uncertain scaling factor reflecting aerosol-cloud
feedbacks (we’ll assume $\alpha = 0.8$).

The code below loads historical and projected radiative forcings (under
the SSP5-8.5 future emissions scenario, which is the most extreme of the
scenarios used to project climate change impacts) from
`data/ERF_ssp585_1750-2500.csv` into a `DataFrame` object and calculates
the non-aerosol and aerosol components of those forcings.

> **Tip**
>
> Look closely at and experiment with the code below: `DataFrames` are a
> common Julia datatype for tabular data, and you may work more with
> them later in the semester or beyond! They are broadly similar to
> `DataFrames` from `pandas` in Python.

In [ ]:
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol

We can plot the aerosol and non-aerosol forcings below.

In [ ]:
t = Int64.(forcings_all[!,"year"]) # Ensure that years are interpreted as integers
p_forcing = plot(; xlabel="Year", ylabel="Radiative Forcing (W/m²)")
plot!(p_forcing, t, forcing_aerosol, label="Aerosol Forcing", color=:blue, linewidth=2)
plot!(p_forcing, t, forcing_non_aerosol, label="Non-Aerosol Forcing", color=:red, linewidth=2)

**In this problem**:

-   Discretize the EBM to produce a simulation model of global mean
    temperatures $T$ over time as a result of total radiative forcings
    $F$.

-   Simulate global mean temperature anomalies (in $^\circ C$ relative
    to 1750) from your model using the historical and SSP5-8.5 radiative
    forcing data. Use an annual time step for the simulation (in
    seconds: $\Delta t = 31,558,152 \mathrm{s}$). You can assume
    $T(0) = 0^\circ C$. Plot the resulting temperature simulation.

-   The climate feedback factor $\lambda$ is one of the key
    uncertainties in projecting future temperatures, even assuming a
    particular scenario of future radiative forcing. Suppose we use the
    following distribution for $\lambda$,
    $$\lambda \sim \text{LogNormal}(\log(2.1), \log(2)/4).$$

    Use Monte Carlo simulation to estimate the expected temperature in
    2100 assuming SSP5-RCP 8.5 radiative forcings (with 95% confidence
    intervals). How does this estimate compare to the value you got from
    your simulation using the expected value of
    $\lambda=2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2)$? How did
    you decide your sample set was sufficient?

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan you identified in Problem 1 is still
    in place for the existing discharges. If the third discharge will
    not be treated, under the original inflow conditions (7.5 mg/L DO),
    how far downstream from the second discharge does this third
    discharge need to be placed to keep the river concentration from
    dropping below 4 mg/L?

## References

List any external references consulted, including classmates.